# Análise de Dados da Fatal Model com PySpark
## Sophia Katze de Paula, Jun/2025

Este notebook apresenta a análise dos dados **users** e **user_transactions**, executando queries SQL via Pandas e PandaSQL, realizando uma simulação Monte Carlo de descontos.

## 1.1 Imports e Setup

Import das bibliotecas necessárias, bem como de ajuste nas tabelas

In [ ]:
# 1. Carregamento Completo e Otimizado com Pandas
# Autora: Sophia Katze de Paula – 2025-06-11

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import sqldf
import os
import sys
import py7zr  

# Configurações gerais
pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid')

#Unzip do Dataset
py7zr.SevenZipFile(r'data/datasets.7z', mode='r').extractall(path=r'data/')

# Carregando os dados 
df_transactions_raw = pd.read_csv(r'data/user_transaction_items_retificado2.csv')
df_users_raw = pd.read_csv(r'data/users.csv')

# Remover espaços dos nomes das colunas e converter para minúsculas
df_transactions_raw.columns = df_transactions_raw.columns.str.replace(' ', '_').str.lower()
df_users_raw.columns = df_users_raw.columns.str.replace(' ', '_').str.lower()


# Exibir os novos nomes das colunas
print("Colunas de df_items:")
print(df_transactions_raw.columns.tolist())

print("\nColunas de df_users:")
print(df_users_raw.columns.tolist())

# Após carregar os DataFrames df_transaction e df_users:
env_init = {
    'transactions_raw': df_transactions_raw,
    'users_raw': df_users_raw
}


In [ ]:
def exec_sql(sql_filename: str, env: dict) -> pd.DataFrame:
    """
    Lê um arquivo .sql da pasta ../sql e executa a query via pandasql.

    Args:
        sql_filename (str): Nome do arquivo .sql (ex: '01_user_status.sql').
        env (dict): Dicionário de DataFrames disponíveis para a query
                    (ex: {'df_transacoes': df_transacoes, 'df_usuarios': df_usuarios}).

    Returns:
        pd.DataFrame: Resultado da consulta.
    """
    # Monta o caminho completo relativo ao notebook em notebooks/
    path = os.path.join('sql', sql_filename)
    with open(path, 'r', encoding='utf-8') as f:
        query = f.read()
    # Executa a query no contexto dos DataFrames fornecidos
    return sqldf(query, env)

# Exemplo de uso:
# env = {'df_transacoes': df_transacoes, 'df_usuarios': df_usuarios}
# df_status = exec_sql('01_user_status.sql', env)

## 1.2 Limpeza dos dados

Tratamento de Null's e Duplicados

In [ ]:
# Print nulls and duplicates for df_users_raw
print("Nulls in df_users_raw:")
print(df_users_raw.isnull().sum())
print("\nDuplicates in df_users_raw:")
print(df_users_raw.duplicated().sum())


In [ ]:
# Print nulls and duplicates for df_transactions_raw  
print("Nulls in df_transactions_raw:")
print(df_transactions_raw.isnull().sum())
print("\nDuplicates in df_transactions_raw:")
print(df_transactions_raw.duplicated().sum())

In [ ]:
#Executamos os SQL de limpeza
df_transactions = exec_sql('clean_user_transactions.sql', env_init)
df_users = exec_sql('clean_users.sql', env_init)

# Após carregar os DataFrames df_transaction e df_users:
env = {
    'user_transactions': df_transactions,
    'users': df_users
}

In [ ]:
# Print nulls and duplicates for df_users
print("Nulls in df_users_raw:")
print(df_users_raw.isnull().sum())
print("\nDuplicates in df_users_raw:")
print(df_users_raw.duplicated().sum())

In [ ]:
# Print nulls and duplicates for df_transactions
print("Nulls in df_transactions:")
print(df_transactions.isnull().sum())
print("\nDuplicates in df_transactions:")
print(df_transactions.duplicated().sum())

## 2. Desafio 1.1: Taxa de Usuários por Status

Qual a proporção de usuários ativos, onboarding, desabilitados e deletados?

In [ ]:
# Executa arquivo sql/01_user_status.sql
df_status = exec_sql('01_user_status.sql', env)
df_status

**Insight:** A saúde da base é medida por X% ativos e Y% banidos, direcionando estratégias de retenção e reengajamento.

## 3. Desafio 1.2: Padrão de Compras e Sazonalidade

Como variam as compras e receita ao longo do tempo?## 3. Desafio 1.2: Padrão de Compras e Sazonalidade

In [ ]:
# Executa arquivo sql/02_daily_sales.sql
df_daily = exec_sql('02_daily_sales.sql', env)

# Converte a coluna para datetime se necessário
df_daily['data'] = pd.to_datetime(df_daily['data'])

#Plotando
plt.figure(figsize=(12,6))
sns.lineplot(data=df_daily, x='data', y='receita')
plt.xticks(rotation=45)
plt.title('Receita Diária')
plt.show()

In [ ]:
import matplotlib.dates as mdates
#O obejtivo é encontrar e indicar no gráfico max e min e respectivas datas

# Encontra o valor máximo e mínimo
max_row = df_daily.loc[df_daily['receita'].idxmax()]
min_row = df_daily.loc[df_daily['receita'].idxmin()]

# Plot principal
plt.figure(figsize=(12,6))
sns.lineplot(data=df_daily, x='data', y='receita')

# Destaques
plt.scatter([max_row['data']], [max_row['receita']], color='green', label='Máximo')
plt.scatter([min_row['data']], [min_row['receita']], color='red', label='Mínimo')

# Anotações
plt.annotate(f"Máx: {max_row['receita']:.0f}\n{max_row['data'].date()}",
             xy=(max_row['data'], max_row['receita']),
             xytext=(max_row['data'], max_row['receita'] * 1.05),
             arrowprops=dict(arrowstyle="->", color='green'),
             color='green')

plt.annotate(f"Mín: {min_row['receita']:.0f}\n{min_row['data'].date()}",
             xy=(min_row['data'], min_row['receita']),
             xytext=(min_row['data'], min_row['receita'] * 1.3),
             arrowprops=dict(arrowstyle="->", color='red'),
             color='red')

# Eixo x formatado
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.xticks(rotation=45)
plt.title('Receita Diária')
plt.legend()
plt.tight_layout()
plt.show()


## 4. Desafio 1.2.1: Agregado por Dia do Mês
Quais dias do mês têm maior volume de vendas?

In [ ]:
# Garante que a coluna 'data' está em datetime
df_daily['data'] = pd.to_datetime(df_daily['data'])

# Extrai o dia do mês
df_daily['dia_do_mes'] = df_daily['data'].dt.day

# Agrega receita média por dia do mês
df_dia_mes = df_daily.groupby('dia_do_mes')['receita'].mean().reset_index()

#Plot
plt.figure(figsize=(14, 6))
sns.barplot(data=df_dia_mes, x='dia_do_mes', y='receita', palette='viridis')

plt.title('Receita Média por Dia do Mês (1 a 31)')
plt.xlabel('Dia do Mês')
plt.ylabel('Receita Média')
plt.xticks(range(0, 31), [str(i+1) for i in range(31)])
plt.tight_layout()
plt.show()


## 4. Desafio 1.2.2: Sazonalidade por Dias da Semana
Há diferença de receita entre dias úteis e finais de semana?

In [ ]:
# Executa SQL de vendas por dia da semana
df_weekday = exec_sql('03_sales_by_weekday.sql', env)

# Ordena corretamente os dias da semana em português
dias_ordenados = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']
df_weekday['dia_semana_nome'] = pd.Categorical(df_weekday['dia_semana_nome'], categories=dias_ordenados, ordered=True)
df_weekday = df_weekday.sort_values('dia_semana_nome')

# Plotagem
plt.figure(figsize=(10,6))
sns.barplot(data=df_weekday, x='dia_semana_nome', y='receita', palette='crest')
plt.title('Receita Total por Dia da Semana')
plt.xlabel('Dia da Semana')
plt.ylabel('Receita Total (mi. R$)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Posteriormente uma outra análise que é possível fazer é de Dias Normais x Feriado

## 4. Desafio 1.3: Usuários Pagantes e ARPU

Quantos usuários pagam e quanto em média gastam?

In [ ]:
# Executa arquivo sql/04_spend_per_user.sql
df_arpu = exec_sql('04_spend_per_user.sql', env)
df_arpu

## 5. Desafio 1.4: Faturamento Mensal

Qual a performance de receita mês a mês?

In [ ]:
# Executa arquivo sql/05_monthly_revenue.sql
df_monthly = exec_sql('05_monthly_revenue.sql', env)

# Criar gráfico de linha mensal
plt.figure(figsize=(14,6))
sns.lineplot(data=df_monthly, x='mes_ano', y='receita', marker='o', linewidth=2)
plt.xticks(rotation=45)
plt.title('Receita Mensal', fontsize=14)
plt.xlabel('Mês/Ano', fontsize=12)
plt.ylabel('Receita Total (mi. R$)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 6. Desafio 2.1: Identificação da Promoção

Quando ocorreu a promoção de 85% de desconto?

In [ ]:
# Executa arquivo sql/06_promotion_period.sql
df_promo = exec_sql('06_promotion_period.sql', env)

df_promo['data']=pd.to_datetime(df_promo['data'])

# Define o início e fim do período promocional
inicio_promocao = df_promo['data'].min()
fim_promocao = df_promo['data'].max()

print(f"Início da promoção: {inicio_promocao.date()}")
print(f"Fim da promoção: {fim_promocao.date()}")

## 7. Desafio 2.2: Impacto da Promoção

Como a receita e o número de transações mudaram?

In [ ]:
# Executa arquivo sql/06_promotion_impact.sql
df_impact = exec_sql('06_promotion_impact.sql', env)
df_impact

## 8. Desafio 2.3: Simulação Monte Carlo de Desconto Ideal

Executamos a simulação para recomendar desconto ideal.

In [ ]:
from scripts import promo_simulation

#from scripts.promo_simulation import simular_receita_descontos, plotar_simulacao
periodo = (df_promo['data'].min(), df_promo['data'].max())
df_sim = simular_receita_descontos(df_transacoes, periodo, list(range(0,91,10)), n_sim=3000)
plotar_simulacao(df_sim)

## 9. Conclusão e Próximos Passos

- Resumo dos principais insights.
- Ações recomendadas: testes de descontos moderados, monitoramento contínuo via Spark.